# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [9]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
# !cat talks.tsv

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
fn = "/Users/linzifan/Desktop/General Research/Website/workspace/ZL_talk_table.csv"

talks = pd.read_csv(fn, sep=",", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,247th meeting of the American Astronomical Soc...,Oral presentation,2026-aas246,"Phoenix, AZ",2026-01-01,"Phoenix, AZ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mid-American Regional Astrophysics Conference ...,Oral presentation,2025-marac,"University of Missouri, Columbia, MO",2025-12-01,"Columbia, MO",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Astronomik Nesnelerin Yapısı ve Evrimi (ANYE) ...,Invited talk,2025-anye,Online,2025-12-01,Online,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Great Lakes Exoplanet Area Meeting (GLEAM) 2025,Poster presentation,2025-gleam,"University of Wisconsin-Madison, Madison, WI",2025-11-01,"Madison, WI",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Solar System in Context,Oral presentation,2025-ss-context,"University of Arizona, Tucson, AZ",2025-09-01,"Tucson, AZ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Washington University Cosmology and AstroPhyic...,Oral presentation,2025-wucap,"WashU Department of Physics, St. Louis, MO",2025-09-01,"St. Louis, MO",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MIT EAPS Planetary Lunch Seminar,Oral presentation,2025-eaps-pls,"MIT EAPS, Cambridge, MA",2025-02-01,"Cambridge, MA",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Gordon Research Conferences: Research at High ...,Poster presentation,2024-grc-hp,"Holderness, NH",2024-07-01,"Holderness, NH",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Gordon Research Seminar: Research at High Pres...,Oral presentation,2024-grs-hp,"Holderness, NH",2024-07-01,"Holderness, NH",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Emerging Researchers in Exoplanet Science Symp...,Oral presentation,2024-eres9,"Cornell University, Ithaca, NY",2024-07-01,"Ithaca, NY",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [13]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [14]:
!ls ../_talks

2019-06-01-2019-eres5.md        2024-07-01-2024-eres9.md
2020-01-01-2020-aas235.md       2024-07-01-2024-grc-hp.md
2021-09-01-2021-elt-workshop.md 2024-07-01-2024-grs-hp.md
2022-03-01-2022-cornell-pls.md  2025-02-01-2025-eaps-pls.md
2022-07-01-2022-grc-hc.md       2025-09-01-2025-ss-context.md
2023-05-01-2023-cmap.md         2025-09-01-2025-wucap.md
2023-06-01-2023-eres8.md        2025-11-01-2025-gleam.md
2023-06-01-2023-grc-ss.md       2025-12-01-2025-anye.md
2023-07-01-2023-uop-workshop.md 2025-12-01-2025-marac.md
2023-12-01-2023-cfa-pizza.md    2026-01-01-2026-aas246.md
2024-05-01-2024-uop-workshop.md


In [15]:
!cat ../_talks/2019-06-01-2019-eres5.md 

---
title: "Emerging Researchers in Exoplanet Science Symposium V"
collection: talks
type: "Poster presentation"
permalink: /talks/2019-06-01-2019-eres5
venue: "Cornell University, Ithaca, NY"
date: 2019-06-01
location: "Ithaca, NY"
---
